# Assignment 1: Dynamic Programming

---

## Task 1) Edit Distances

Implement the [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) and [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit) distances and compute them for the for the following word pairs.
It may help to compute them by hand first.

<img src = "./assets/97090.jpg" width="33%" /> <img src = "./assets/97222.jpg" width="33%" /> <img src = "./assets/97669.jpg" width="33%" />

Aside from computing the distance (which is a scalar), do the backtrace and compute the edit transcript (and thus alignment).

---

In [1]:
WORD_PAIRS = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [2]:
def hamming(s1: str, s2: str) -> int:
    """
    Compute the hamming distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int.
    """
    # Hint: Think about how you can deal with unequal word lengths.
    
    ### YOUR CODE HERE

    dist = 0
    longer, shorter = (s1, s2) if len(s1) > len(s2) else (s2, s1)
    
    for i in range(0, len(shorter)):
        if longer[i] != shorter[i]:
            dist+=1
            
    if len(longer) != len(shorter):
        dist += len(longer) - len(shorter)
            
    return dist
    
    ### END YOUR CODE

In [3]:
for wordpair in WORD_PAIRS:
    dist = hamming(s1=wordpair[0], s2=wordpair[1])
    print("hamming('{}', '{}') = {}".format(
        wordpair[0], wordpair[1], dist
    ))

### EXPECTED
# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
hamming('kühler schrank', 'schüler krank') = 13
hamming('the longest', 'longest day') = 11
hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
hamming('gurken schaben', 'schurkengaben') = 14


In [4]:
def levenshtein(s1: str, s2: str, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}) -> (int, str):
    """
    Compute the levenshtein (edit) distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int and edit transcript as string.
    """
    # Hint: The probably most intuitive approach is bottom-up.
    
    ### YOUR CODE HERE
    
    cols = len(s1) + 1
    rows = len(s2) + 1
    transcript_matrix = [["" for _ in range(cols)] for _ in range(rows)]
    
#    if len(s1) == 0:
#        return len(s2)
#    
#    if len(s2) == 0:
#        return len(s1) 
    
    dist_matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    
    for i in range(1, rows):
        dist_matrix[i][0] = i
        transcript_matrix[i][0] = transcript_matrix[i-1][0] + 'I'
        
    for j in range(1, cols):
        dist_matrix[0][j] = j
        transcript_matrix[0][j] = transcript_matrix[0][j-1] + 'D'
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s1[col - 1] == s2[row - 1]:
                cost = 0
                operation = 'M'
                
            else:
                cost = 1
                operation = "R"
                
            replace = dist_matrix[row-1][col-1] + cost
            insert = dist_matrix[row][col-1] + 1
            delete = dist_matrix[row-1][col] + 1
                
            min_cost = min(replace,insert,delete)
                
            dist_matrix[row][col]= min_cost
                
            if min_cost == delete:
                transcript_matrix[row][col] = transcript_matrix[row-1][col] + 'I'
            elif min_cost == insert:
                transcript_matrix[row][col] = transcript_matrix[row][col-1] + 'D'
            else:
                transcript_matrix[row][col] = transcript_matrix[row-1][col-1] + operation

    return dist_matrix[-1][-1], transcript_matrix[-1][-1]
    ### END YOUR CODE

In [5]:
for wordpair in WORD_PAIRS:
    dist, transcript = levenshtein(s1=wordpair[0], s2=wordpair[1])
    print("levenshtein('{}', '{}') = {} ({})".format(
        wordpair[0], wordpair[1], dist, transcript
    ))

### EXPECTED
# levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
# levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
# levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
# levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
# levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)

levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (MMDMMMMRMMMMMIMMMM)
levenshtein('kühler schrank', 'schüler krank') = 6 (RRMIMMMMRDDMMMM)
levenshtein('the longest', 'longest day') = 8 (DDDDMMMMMMMIIII)
levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (RMMMMMMMMMMRMRRM)
levenshtein('gurken schaben', 'schurkengaben') = 7 (RIIMMMMMRDDDMMMM)


---

## Task 2) Basic Spelling Correction using Edit Distance

For spelling correction, we will use prior knowledge, to put _some_ learning into our system.

The underlying idea is the _Noisy Channel Model_, that is: The user _intends_ to write a word `w`, but through some noise in the process, happens to type the word `x`.

The correct word $\hat{w}$ is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

1. The candidates $V$ can be obtained from a _vocabulary_.
2. The probability $P(w)$ of a word $w$ can be _learned (counted) from data_.
3. The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a _heuristic_ that relates to the edit distance, e.g. rank by distance.

You may use [Peter Norvig's count_1w.txt](http://norvig.com/ngrams/) file, [local mirror](res/count_1w.tar.bz2).
Note that it may help to restrict to the first 10k words to get started.

---

In [6]:
EXAMPLES = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

In [7]:
### TODO: Prepare the vocabulary

### YOUR CODE HERE

def load_word_frequencies(file_path: str, limit=10000) -> dict:
    """
    Load word frequencies from a file into a dictionary.
    """
    word_frequencies = {}
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i >= limit:  
                break
            word, freq = line.split()
            word_frequencies[word] = int(freq)
    return word_frequencies

word_freqs = load_word_frequencies("data/count_1w.txt")

print([(word, freq) for word, freq in list(word_freqs.items())[:3]])


### END YOUR CODE

[('the', 23135851162), ('of', 13151942776), ('and', 12997637966)]


In [8]:
import math

def probability_x_given_w(x, w, lambda_param=0.5):
    """Calculate the probability P(x|w) using edit distance with exponential decay."""
    distance = levenshtein(x, w)[0]
    return math.exp(-lambda_param * distance)

def suggest(w: str, dist_fn, max_cand=5) -> list:
    candidates = word_freqs.keys()
    # Ensure dist_fn returns a tuple where the first item is the distance
    distances = [(candidate, dist_fn(w, candidate)[0]) for candidate in candidates]
    distances.sort(key=lambda x: x[1])
    
    scored_candidates = []
    for rank, (candidate, distance) in enumerate(distances, start=1):
        frequency = word_freqs.get(candidate, 0)
        score = (1 / rank) * frequency
        scored_candidates.append((candidate, distance, score))
    
    scored_candidates.sort(key=lambda x: (x[1], -x[2]))

    return scored_candidates[:max_cand]
    ### END YOUR CODE

In [9]:
# How does your suggest function behave with levenshtein distance?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=levenshtein, max_cand=3)
    print("{} {}".format(word, suggestions))

### EXPECTED
### Notice: your scores may vary!
# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

pirates [('pirates', 0, 6531487.0), ('rates', 2, 68544769.0), ('plates', 2, 4736699.666666666)]
pirutes [('pirates', 1, 6531487.0), ('minutes', 2, 48121104.5), ('viruses', 2, 2928857.6666666665)]
continoisly [('continuously', 2, 5610397.0), ('continuity', 3, 3022781.0), ('continually', 3, 1759573.6666666665)]


### Food for Thought

- How would you modify the implementation so that it works fast for large lexica (eg. the full file above)?
- How would you modify the implementation so that it works "while typing" instead of at the end of a word?
- How do you handle unknown/new words?

Large Lexica:
- Prefilter (e.g. word-length, first couple of letters)
- Caching
- Efficient data structures

While typing:
- only look at partial words 

New words:
- embeddings?

---

## Task 3) Needleman-Wunsch: Keyboard-aware Auto-Correct

In the parts 1 and 2 above, we applied uniform cost to all substitutions.
This does not really make sense if you look at a keyboard: the QWERTY layout will favor certain substitutions (eg. _a_ and _s_), while others are fairly unlikely (eg. _a_ and _k_).

Implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) which is very similar to the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), however it doesn't _minimize the cost_ but _maximizes the similarity_.
For a good measure of similarity, implement a metric that computes a meaningful weight for a given character substitution.

---

In [10]:
import numpy as np

def keyboardsim(s1: str, s2: str) -> float:
    ### YOUR CODE HERE
    
    keyboard_layout = {
        # Top row
        'q': (0, 0.0), 'w': (0, 1.0), 'e': (0, 2.0), 'r': (0, 3.0), 't': (0, 4.0),
        'z': (0, 5.0), 'u': (0, 6.0), 'i': (0, 7.0), 'o': (0, 8.0), 'p': (0, 9.0),
        # Middle row
        'a': (1, 0.5), 's': (1, 1.5), 'd': (1, 2.5), 'f': (1, 3.5), 'g': (1, 4.5),
        'h': (1, 5.5), 'j': (1, 6.5), 'k': (1, 7.5), 'l': (1, 8.5),
        # Bottom row
        'y': (2, 1.0), 'x': (2, 2.0), 'c': (2, 3.0), 'v': (2, 4.0), 'b': (2, 5.0),
        'n': (2, 6.0), 'm': (2, 7.0),
    }

    pos1 = keyboard_layout.get(s1.lower())
    pos2 = keyboard_layout.get(s2.lower())

    if not pos1 or not pos2:
        return 0.0

    # Eucild. distance
    dist = ((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2) ** 0.5

    # lower distance -> higher score
    similarity = 1 / (1 + dist)
    return similarity
    
    ### END YOUR CODE


def nw(s1: str, s2: str, d: float, sim_fn) -> float:
    """
    Apply needleman-wunsch algorithm.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    d: Gap penalty score.
    sim_fn: Similarity function to use.

    Returns the score as float.
    """
    ### YOUR CODE HERE
    
    D = np.zeros((len(s1) + 1, len(s2) + 1), dtype=float) 
    for i in range(1, len(s2) + 1):
        D[0, i] = i * d
    for i in range(1, len(s1) + 1):
        D[i, 0] = i * d

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            cs = D[i-1, j-1] + sim_fn(s1[i-1], s2[j-1])
            cd = D[i-1, j] + d
            ci = D[i, j-1] + d
            D[i, j] = max(cs, cd, ci)

    return D[len(s1), len(s2)]

    ### END YOUR CODE


def nw_based_dist(s1: str, s2: str) -> (int, str):
    """
    Compute the needleman-wunsch distance between two strings.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    
    Returns the distance as int and <unsupported> string.
    """
    ### YOUR CODE HERE
    
    score = nw(s1, s2, -1, keyboardsim)
    dist = round(len(s1) - score)

    return dist, "<unsupported>"
    # Hint: return dist, "<unsupported>"
    
    ### END YOUR CODE

In [11]:
# How does your suggest function behave with nw and a keyboard-aware similarity?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=nw_based_dist, max_cand=3)
    print("{} {}".format(word, suggestions))

pirates [('pirates', 0, 6531487.0), ('phrases', 1, 3298311.0), ('updated', 2, 36501588.666666664)]
pirutes [('pirates', 1, 6531487.0), ('pictures', 2, 107498959.0), ('limited', 2, 35384989.666666664)]
continoisly [('continuously', 2, 5610397.0), ('continually', 2, 2639360.5), ('continuity', 3, 2015187.3333333333)]


In [12]:
test = nw("pirate", "pxrate", -1, keyboardsim)
print(test)

5.156613028826232


### Food for Thought

- What could be better heuristics for similarity resp. cost of substitution than the one above?
- What about capitalization, punctiation and special characters?
- What about swipe-to-type?